# 00. 事前準備と環境設定

## 概要

このノートブックでは、SharePoint から Azure AI Search へデータを取り込むハンズオンに必要な環境を準備します。

### アーキテクチャ概観

```
┌─────────────────────┐
│  SharePoint Online  │
│   (Documents)       │
└──────────┬──────────┘
           │
           ├─────────────────────┬───────────────────────┐
           │                     │                       │
    ┌──────▼────────┐   ┌───────▼────────┐   ┌─────────▼─────────┐
    │  Pattern A    │   │   Pattern B    │   │  Azure Document   │
    │  Graph API    │   │  SP Indexer    │   │  Intelligence     │
    │  + Python     │   │  (Native)      │   │  (Optional)       │
    └──────┬────────┘   └───────┬────────┘   └─────────┬─────────┘
           │                     │                       │
           └─────────────────────┴───────────────────────┘
                                 │
                        ┌────────▼──────────┐
                        │  Azure AI Search  │
                        │  (Index + Vector) │
                        └───────────────────┘
```

### 2つのパターン

**Pattern A: Graph API 取り込み**
- Microsoft Graph API でファイルとメタデータを取得
- Python で本文抽出・チャンク分割・埋め込み生成
- 柔軟なカスタマイズが可能
- 差分同期は Graph delta API を使用

**Pattern B: SharePoint Indexer**
- Azure AI Search の組み込み SharePoint インデクサを使用
- 設定だけで自動取り込み
- スケジュール実行で自動差分同期
- 運用が簡単

---

## 必要なリソース

### Azure リソース

1. **Azure AI Search** (必須)
   - Basic 以上のティア推奨
   - ベクター検索を使う場合は Standard 以上

2. **Azure OpenAI** (任意)
   - 埋め込み生成用 (text-embedding-ada-002 または text-embedding-3-large)
   - 統合ベクタライザーを使う場合は必要

3. **Azure Document Intelligence** (任意)
   - 高精度テキスト抽出・OCR用
   - 複雑なレイアウトや画像多用の文書に有効

### SharePoint

- テスト用サイトとドキュメントライブラリ
- 例: `/sites/ai-search-handson`
- 各種フォーマットのサンプルファイル (PDF, Word, Excel, PowerPoint)

### Azure AD アプリ登録 (Pattern A 用)

- **アプリケーション権限**:
  - `Sites.Read.All` または `Sites.ReadWrite.All`
  - `Files.Read.All` または `Files.ReadWrite.All`
- 管理者の同意が必要
- クライアントID、テナントID、シークレットを取得

### 重要な注意事項

⚠️ **セキュリティ**:
- `.env` ファイルは Git にコミットしない
- 本番環境では Azure Key Vault を使用
- ログに機密情報を出力しない

⚠️ **コスト**:
- Document Intelligence の使用量
- OpenAI の埋め込み生成トークン
- Search のストレージとクエリ

⚠️ **API 制限**:
- Graph API: スロットリング (429 エラー) に注意
- リトライロジックの実装を推奨

## 1. 必要なライブラリのインストール

requirements.txt に記載されているパッケージをインストールします。

In [ ]:
# requirements.txt からインストール
!pip install -q -r ../requirements.txt
print("✅ ライブラリのインストールが完了しました")

## 2. 環境変数の読み込みと検証

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# .env ファイルの読み込み
env_path = Path("..") / ".env"
if not env_path.exists():
    print("❌ .env ファイルが見つかりません")
    print("   .env.example をコピーして .env を作成し、設定を記入してください")
    raise FileNotFoundError(".env file not found")

load_dotenv(dotenv_path=env_path, override=True)
print("✅ .env ファイルを読み込みました")

## 3. 必須設定の検証

In [ ]:
def validate_env_vars():
    """環境変数の検証"""
    required_vars = {
        "Azure AI Search": [
            "AZURE_SEARCH_ENDPOINT",
            "AZURE_SEARCH_API_KEY",
            "AZURE_SEARCH_INDEX_NAME"
        ],
        "Microsoft Graph (Pattern A)": [
            "GRAPH_TENANT_ID",
            "GRAPH_CLIENT_ID",
            "GRAPH_CLIENT_SECRET"
        ],
        "SharePoint": [
            "SP_HOSTNAME",
            "SP_SITE_PATHS"
        ]
    }
    
    optional_vars = {
        "Azure OpenAI (Optional)": [
            "AZURE_OPENAI_ENDPOINT",
            "AZURE_OPENAI_API_KEY",
            "AZURE_OPENAI_EMBEDDING_DEPLOYMENT"
        ],
        "Document Intelligence (Optional)": [
            "AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT",
            "AZURE_DOCUMENT_INTELLIGENCE_API_KEY"
        ]
    }
    
    all_ok = True
    
    # 必須変数のチェック
    print("=" * 60)
    print("必須設定の検証")
    print("=" * 60)
    for category, vars_list in required_vars.items():
        print(f"\n[{category}]")
        for var in vars_list:
            value = os.getenv(var)
            if value and value != f"your-{var.lower().replace('_', '-')}":
                print(f"  ✅ {var}: 設定済み")
            else:
                print(f"  ❌ {var}: 未設定")
                all_ok = False
    
    # オプション変数のチェック
    print("\n" + "=" * 60)
    print("オプション設定の検証")
    print("=" * 60)
    for category, vars_list in optional_vars.items():
        print(f"\n[{category}]")
        for var in vars_list:
            value = os.getenv(var)
            if value and value != f"your-{var.lower().replace('_', '-')}":
                print(f"  ✅ {var}: 設定済み")
            else:
                print(f"  ⚠️  {var}: 未設定 (オプション)")
    
    print("\n" + "=" * 60)
    if all_ok:
        print("✅ すべての必須設定が完了しています")
    else:
        print("❌ 一部の必須設定が未完了です。.env ファイルを確認してください")
    print("=" * 60)
    
    return all_ok

validation_result = validate_env_vars()

## 4. SDK のインポートとバージョン確認

In [ ]:
import sys
import importlib.metadata

# 主要パッケージのバージョン確認
packages = [
    "azure-search-documents",
    "azure-identity",
    "msal",
    "requests",
    "python-dotenv",
    "pdfminer.six",
    "python-docx",
    "openai"
]

print("=" * 60)
print("インストール済みパッケージのバージョン")
print("=" * 60)
print(f"Python: {sys.version.split()[0]}")
print()

for package in packages:
    try:
        version = importlib.metadata.version(package)
        print(f"✅ {package:30s} {version}")
    except importlib.metadata.PackageNotFoundError:
        print(f"❌ {package:30s} Not installed")

print("=" * 60)

## 5. Azure AI Search への接続テスト

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient

try:
    search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
    search_key = os.getenv("AZURE_SEARCH_API_KEY")
    
    if not search_endpoint or not search_key:
        raise ValueError("Azure Search の設定が不足しています")
    
    # 接続テスト
    index_client = SearchIndexClient(
        endpoint=search_endpoint,
        credential=AzureKeyCredential(search_key)
    )
    
    # サービス統計の取得
    stats = index_client.get_service_statistics()
    
    print("=" * 60)
    print("✅ Azure AI Search への接続成功")
    print("=" * 60)
    print(f"エンドポイント: {search_endpoint}")
    print(f"ストレージ使用量: {stats['counters']['storage_size']:,} bytes")
    print(f"インデックス数: {stats['counters']['index_counter']['usage']} / {stats['counters']['index_counter']['quota']}")
    print("=" * 60)
    
except Exception as e:
    print(f"❌ Azure AI Search への接続エラー: {str(e)}")
    print("   エンドポイントと API キーを確認してください")

## 6. Microsoft Graph API への接続テスト (Pattern A 用)

In [ ]:
import msal
import requests

try:
    tenant_id = os.getenv("GRAPH_TENANT_ID")
    client_id = os.getenv("GRAPH_CLIENT_ID")
    client_secret = os.getenv("GRAPH_CLIENT_SECRET")
    
    if not all([tenant_id, client_id, client_secret]):
        raise ValueError("Microsoft Graph の設定が不足しています")
    
    # MSAL でトークン取得
    authority = f"https://login.microsoftonline.com/{tenant_id}"
    app = msal.ConfidentialClientApplication(
        client_id,
        authority=authority,
        client_credential=client_secret
    )
    
    result = app.acquire_token_for_client(
        scopes=["https://graph.microsoft.com/.default"]
    )
    
    if "access_token" in result:
        # テスト API 呼び出し
        headers = {"Authorization": f"Bearer {result['access_token']}"}
        response = requests.get(
            "https://graph.microsoft.com/v1.0/sites?$top=1",
            headers=headers
        )
        
        if response.status_code == 200:
            print("=" * 60)
            print("✅ Microsoft Graph API への接続成功")
            print("=" * 60)
            print(f"テナントID: {tenant_id}")
            print(f"クライアントID: {client_id}")
            print("アクセストークンの取得に成功しました")
            print("=" * 60)
        else:
            print(f"❌ Graph API 呼び出しエラー: {response.status_code}")
            print(f"   レスポンス: {response.text[:200]}")
    else:
        error = result.get("error_description", result.get("error", "Unknown error"))
        print(f"❌ トークン取得エラー: {error}")
        
except Exception as e:
    print(f"❌ Microsoft Graph への接続エラー: {str(e)}")
    print("   アプリ登録の設定と権限を確認してください")

## 7. Azure OpenAI への接続テスト (オプション)

In [ ]:
from openai import AzureOpenAI

try:
    openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    openai_key = os.getenv("AZURE_OPENAI_API_KEY")
    embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
    
    if not all([openai_endpoint, openai_key, embedding_deployment]):
        print("⚠️  Azure OpenAI の設定がスキップされました (オプション)")
    else:
        # OpenAI クライアント作成
        client = AzureOpenAI(
            azure_endpoint=openai_endpoint,
            api_key=openai_key,
            api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")
        )
        
        # テスト埋め込み生成
        response = client.embeddings.create(
            input="テスト",
            model=embedding_deployment
        )
        
        print("=" * 60)
        print("✅ Azure OpenAI への接続成功")
        print("=" * 60)
        print(f"エンドポイント: {openai_endpoint}")
        print(f"デプロイメント: {embedding_deployment}")
        print(f"埋め込み次元: {len(response.data[0].embedding)}")
        print("=" * 60)
        
except Exception as e:
    print(f"⚠️  Azure OpenAI への接続エラー: {str(e)}")
    print("   ベクター検索を使わない場合は問題ありません")

## 8. 設定サマリー

In [ ]:
print("=" * 60)
print("設定サマリー")
print("=" * 60)
print(f"\n📁 SharePoint サイト: {os.getenv('SP_HOSTNAME')}{os.getenv('SP_SITE_PATHS')}")
print(f"📚 対象ライブラリ: {os.getenv('SP_LIBRARY_NAMES', 'Documents')}")
print(f"\n🔍 検索インデックス名: {os.getenv('AZURE_SEARCH_INDEX_NAME')}")
print(f"\n📄 サポート拡張子: {os.getenv('SUPPORTED_EXTENSIONS')}")
print(f"✂️  チャンクサイズ: {os.getenv('CHUNK_SIZE')} 文字")
print(f"🔄 チャンク重複: {os.getenv('CHUNK_OVERLAP')} 文字")
print(f"📊 最大処理ファイル数: {os.getenv('MAX_FILES_TO_PROCESS')}")
print("\n" + "=" * 60)
print("\n✅ 環境設定が完了しました!")
print("\n次のステップ:")
print("  - Pattern A: A1_graph_ingest_setup.ipynb に進む")
print("  - Pattern B: B1_sp_indexer_setup.ipynb に進む")
print("=" * 60)